## title

In [ ]:
(define (make-wire)
  (let ((signal-value 0) (action-procedures '()))
    (define (set-my-signal! new-value)
      (if (not (= signal-value new-value))
	  (begin (set! signal-value new-value)
		 (call-each action-procedures))
	  'done))
    (define (accept-action-procedure! proc)
      (set! action-procedures (cons proc action-procedures))
      (proc))

    (define (dispatch m)
      (cond ((eq? m 'get-signal) signal-value)
	    ((eq? m 'set-signal!) set-my-signal!)
	    ((eq? m 'add-action!) accept-action-procedure!)
	    (else (error "Unknown operation --WIRE" m))))
    dispatch))

(define (call-each procedures)
  (if (null? procedures)
      'done
      (begin
	((car procedures))
	(call-each (cdr procedures)))))

(define (after-delay delay action)
  (add-to-agenda! (+ delay (current-time the-agenda))
		  action
		  the-agenda))

(define (propagate)
 ; (display "handling: " ) (display the-agenda) (newline) 
  (if (empty-agenda? the-agenda)
      'done
      (let ((first-item (first-agenda-item the-agenda)))
	(first-item)
	(remove-first-agenda-item! the-agenda)
	(propagate))))

(define (get-signal wire)
  (wire 'get-signal))

(define (set-signal! wire new-value)
  ((wire 'set-signal!) new-value))

(define (add-action! wire action-procedure)
  ((wire 'add-action!) action-procedure))



(define (half-adder a b s c)
  (let ((d (make-wire)) (e (make-wire)))
    (or-gate a b d)
    (and-gate a b c)
    (inverter c e)
    (and-gate d e s)
  ;  (probe 'inter-a a)
   ; (probe 'inter-b b)
   ; (probe 'inter-c c)
   ; (probe 'inter-d d)
   ; (probe 'inter-e e)
   ; (probe 'inter-s s)


    'ok))




(define (full-adder a b c-in sum c-out)
  (let ((s (make-wire))
	(c1 (make-wire))
	(c2 (make-wir)))
    (half-adder b c-in s c1)
    (half-adder a s sum c2)
    (or-gate c1 c2 c-out)
    'ok))

(define (inverter input output)
  (define (invert-input)
    (let ((new-value (logical-not (get-signal input))))
      (after-delay inverter-delay
		   (lambda ()
		     (set-signal! output new-value)))))
  (add-action! input invert-input)
  'ok)

(define (logical-not s)
  (cond ((= s 0 ) 1)
	((= s 1) 0)
	(else (error "Invalid signal" s))))

(define (and-gate a1 a2 output)
  (define (and-action-procedure)
    (let ((new-value
	   (logical-and (get-signal a1) (get-signal a2))))
      (after-delay and-gate-delay
		   (lambda ()
		     (set-signal! output new-value)))))
  (add-action! a1 and-action-procedure)
  (add-action! a2 and-action-procedure)
  'ok)

(define (logical-and a1 a2)
  (cond ((and (not (= a1 0))
	     (not (= a1 1))) (error "Invalid signal" a1))
	((and (not (= a2 0))
	     (not (= a2 1))) (error "Invalid signal" a2))
	((and (= a1 1) (= a2 1)) 1)
	(else 0)))

(define (or-gate a1 a2 output)
  (define (or-action-procedure)
    (let ((new-value
	   (logical-or (get-signal a1) (get-signal a2))))
      (after-delay or-gate-delay
		   (lambda ()
		     (set-signal! output new-value)))))
  (add-action! a1 or-action-procedure)
  (add-action! a2 or-action-procedure)
  'ok)


  

(define (logical-or a1 a2)
  (cond ((and (not (= a1 0))
	     (not (= a1 1))) (error "Invalid signal" a1))
	((and (not (= a2 0))
	     (not (= a2 1))) (error "Invalid signal" a2))
	((or (= a1 1) (= a2 1)) 1)
	(else 0)))

(define (make-time-segment time queue)
  (cons time queue))

(define (segment-time s) (car s))

(define (segment-queue s) (cdr s))


(define (make-agenda) (list 0))

(define (current-time agenda) (car agenda))

(define (set-current-time! agenda time)
  (set-car! agenda time))

(define (segments agenda) (cdr agenda))

(define (set-segments! agenda segments)
  (set-cdr! agenda segments))

(define (first-segment agenda) (car (segments agenda)))

(define (rest-segments agenda) (cdr (segments agenda)))

(define (empty-agenda? agenda)
  (null? (segments agenda)))

(define (add-to-agenda! time action agenda)
  (define (belongs-before? segments)
    (or (null? segments)
	(< time (segment-time (car segments)))))
  (define (make-new-time-segment time action)
    (let ((q (make-queue)))
      (insert-queue! q action)
      (make-time-segment time q )))
  (define (add-to-segments! segments)
    (if (= (segment-time (car segments)) time)
	(insert-queue! (segment-queue (car segments))
		       action)
	(let ((rest (cdr segments)))
	  (if (belongs-before? rest)
	      (set-cdr!
	       segments
	       (cons (make-new-time-segment time action)
		     (cdr segments)))
	      (add-to-segments! rest)))))
  (let ((segments (segments agenda)))
    (if (belongs-before? segments)
	(set-segments!
	 agenda
	 (cons (make-new-time-segment time action)
	       segments))
	(add-to-segments! segments))))


(define (remove-first-agenda-item! agenda)
  (let ((q (segment-queue (first-segment agenda))))
    (delete-queue! q)
    (if (empty-queue? q)
	(set-segments! agenda (rest-segments agenda)))))

(define (first-agenda-item agenda)
  (if (empty-agenda? agenda)
      (error "Agenda is empty -- FIRST-AGENDA-ITEM")
      (let ((first-seg (first-segment agenda)))
	(set-current-time! agenda (segment-time first-seg))
	(front-queue (segment-queue first-seg)))))


(define (probe name wire)
  (add-action! wire
	       (lambda ()
		 (newline)
		 (display name)
		 (display " ")
		 (display (current-time the-agenda))
		 (display " New-value = ")
		 (display (get-signal wire)))))

(define the-agenda (make-agenda))

(define inverter-delay 2)
(define and-gate-delay 3)
(define or-gate-delay 5)

(define (front-ptr queue) (car queue))

(define (rear-ptr queue) (cdr queue))

(define (set-front-ptr! queue item) (set-car! queue item))

(define (set-rear-ptr! queue item) (set-cdr! queue item))

(define (empty-queue? queue) (null? (front-ptr queue)))

(define (make-queue) (cons '() '()))

(define (front-queue queue)
  (if (empty-queue? queue)
      (error "FRONT called with an empty queue" queue)
      (car (front-ptr queue))))

(define (insert-queue! queue item)
  (let ((new-pair (cons item '())))
    (cond ((empty-queue? queue)
	   (set-front-ptr! queue new-pair)
	   (set-rear-ptr! queue new-pair)
	   queue)
	  (else
	   (set-cdr! (rear-ptr queue) new-pair)
	   (set-rear-ptr! queue new-pair)
	   queue))))

(define (delete-queue! queue)
  (cond ((empty-queue? queue)
	 (error "DELETE! called with an empty queue" queue))
	(else
	 (set-front-ptr! queue (cdr (front-ptr queue)))))
	 queue)

(define (print-queue queue)
  (cond ((null? queue)
	 (error "print-queue called with an incorrect data" queue))
	(else
	 (display (car queue)))))

(define (full-adder a b c-in sum c-out)
  (let ((s (make-wire))
	(c1 (make-wire))
	(c2 (make-wire)))
    (half-adder b c-in s c1)
    (half-adder a s sum c2)
    (or-gate c1 c2 c-out)
    'ok))

(define (ripple-carry-adder a-list b-list s-list c)
  (define (inter-carry-adder a-inter b-inter s-inter c-inter)
   ; (display "handling a-inter: ") (display a-inter) (newline)
    (if (null? a-inter)
	c-inter
	(let ((new-c (make-wire)))
	  (full-adder (car a-inter) (car b-inter) c-inter (car s-inter) new-c)
	  (inter-carry-adder (cdr a-inter) (cdr b-inter) (cdr s-inter) new-c))))
  (if (not (= (length a-list) (length b-list) (length s-list)))
      (error "incorrect input parameters" (list a-list b-list s-list))
      (inter-carry-adder a-list b-list s-list c)))

(define (start-test-half-adder)

  (display "half adder testing") (newline)
  
  (define t-a (make-wire))
  (define t-b (make-wire))
  (define t-sum (make-wire))
  (define t-c-out (make-wire))
  
  
  (half-adder t-a t-b t-sum t-c-out)

  (probe 't-a t-a)
  (probe 't-b t-b)
  (probe 't-sum t-sum)
  (probe 't-c-out t-c-out)
 
  (newline)
  (display "ready to set!") (newline)
  (set-signal! t-a 1)
  (propagate)
  
  (set-signal! t-b 0)
 
  (propagate)
 


)



(define (start-test-full-adder)

  (display "full adder testing") (newline)
  
  (define t-a (make-wire))
  (define t-b (make-wire))
  (define t-c-in (make-wire))
  (define t-sum (make-wire))
  (define t-c-out (make-wire))
  
  
  (full-adder t-a t-b t-c-in t-sum t-c-out)

  (probe 't-a t-a)
  (probe 't-b t-b)
  (probe 't-c-in t-c-in)
  (probe 't-sum t-sum)
  (probe 't-c-out t-c-out)

  (set-signal! t-a 1)
  (set-signal! t-b 1)
  (set-signal! t-c-in 0)

  (propagate)
 


)

(define (start-test-3-30)

 
  (define a-1 (make-wire))
  (define a-2 (make-wire))
  (define b-1 (make-wire))
  (define b-2 (make-wire))
  
  (define s-1 (make-wire))
  (define s-2 (make-wire))
  (define c (make-wire))

  (define a-list (list a-1 a-2))
  (define b-list (list b-1 b-2))
  (define s-list (list s-1 s-2))

  (probe 'a-1 a-1)
  (probe 'a-2 a-2)
  (probe 'b-1 b-1)
  (probe 'b-2 b-2)
  (probe 's-1 s-1)
  (probe 's-2 s-2)

 
  (define c-out (ripple-carry-adder a-list b-list s-list c))

  (probe 'c-out c-out)

  (set-signal! a-1 1)
  (set-signal! a-2 1)
  (set-signal! b-1 1)
  (set-signal! b-2 1)
  (set-signal! c 0)

  (propagate)
 ; (propagate)
 ; (propagate)

  (newline)
  (display "s1 is: ") (display (get-signal s-1)) (newline)
  (display "s2 is: ") (display (get-signal s-2)) (newline)
  (display "c-out is: ") (display (get-signal c-out)) (newline)


)


(define (start-test-3-31)
  (display "if we don't exe the procedure when we insert it")
  (newline)
  (display "the machine can't response to the init value of the wire")
  (newline))

(define (display-agenda)
  (display "the agenda is:------------") (newline)
  (display (map (lambda (x) (list x)) the-agenda))
  (newline))

(define (start-test-3-32)
  (define a1 (make-wire))
  (define a2 (make-wire))
  (define output (make-wire))
  
  (and-gate a1 a2 output)

  (set-signal! a2 1)
  (propagate)
  
  (set-signal! a1 1)
  (set-signal! a2 0)

  (display-agenda)

  (display "In this scenario, we got (set-signal! output 1) and (set-signal! output 0) in the agenda")
  (newline)
  (display "if we run them in sequence, we got right value 0")
  (newline)
  (display "if we run them constranstly, we got the wrong value 1")
  (newline)
)
  